In [ ]:
!pip install ../input/wheat-detection/torch-1.5.0cu101-cp37-cp37m-linux_x86_64.whl
!pip install ../input/wheat-detection/torchvision-0.6.0cu101-cp37-cp37m-linux_x86_64.whl

In [ ]:
!pip install ../input/wheat-detection/Pillow-7.2.0-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/wheat-detection/pycocotools-2.0.1-cp37-cp37m-linux_x86_64.whl
!pip install ../input/wheat-detection/yacs-0.1.7-py3-none-any.whl
!pip install ../input/wheat-detection/fvcore-0.1.1.post20200716-py3-none-any.whl

In [ ]:
!pip install ../input/wheat-detection/detectron2-0.2cu101-cp37-cp37m-linux_x86_64.whl

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger

setup_logger()

In [ ]:
from detectron2.data import datasets, DatasetCatalog, MetadataCatalog,\
build_detection_train_loader, build_detection_test_loader
import pandas as pd

DIR_INPUT = '../input/global-wheat-detection'
sub_df = pd.read_csv(f'{DIR_INPUT}/sample_submission.csv')
data_dir = '../input/global-wheat-detection/test'
data_set_prefix = 'wheat'

for d in ["train", "test"]:
    DatasetCatalog.register(
        f"{data_set_prefix}_{d}",
        lambda d=d: wheat_dataset(
            tsub_df,
            data_dir,
            False,
        ),
    )


In [ ]:
import cv2
from tqdm.notebook import tqdm
import os

def wheat_dataset(df, folder, is_train):
    unique_img_names = df["image_id"].unique().tolist()  # Take unique image names
    df_group = df.groupby("image_id")  # Group the training by the image ids
    dataset_dicts = []

    for img_id, img_name in enumerate(tqdm(unique_img_names)):
        img_group = df_group.get_group(img_name)  # Take all annotations for an image
        img_path = os.path.join(folder, img_name + ".jpg")  # Create path for an image
        img = cv2.imread(img_path)
        h, w = img.shape[:2]

        record = dict()
        record["file_name"] = img_path
        record["image_id"] = img_id
        record["height"] = int(h)
        record["width"] = int(w)

        dataset_dicts.append(record)

    return dataset_dicts

In [ ]:
class BaseWheatTTA:
    """ author: @shonenkov """
    image_size = 1024

    def augment(self, image):
        raise NotImplementedError
    
    def batch_augment(self, images):
        raise NotImplementedError
    
    def deaugment_boxes(self, boxes):
        raise NotImplementedError

class TTAHorizontalFlip(BaseWheatTTA):
    """ author: @shonenkov """

    def augment(self, image):
        return image.flip(1)
    
    def batch_augment(self, images):
        return images.flip(2)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [1,3]] = self.image_size - boxes[:, [3,1]]
        return boxes

class TTAVerticalFlip(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return image.flip(2)
    
    def batch_augment(self, images):
        return images.flip(3)
    
    def deaugment_boxes(self, boxes):
        boxes[:, [0,2]] = self.image_size - boxes[:, [2,0]]
        return boxes
    
class TTARotate90(BaseWheatTTA):
    """ author: @shonenkov """
    
    def augment(self, image):
        return torch.rot90(image, 1, (1, 2))

    def batch_augment(self, images):
        return torch.rot90(images, 1, (2, 3))
    
    def deaugment_boxes(self, boxes):
        res_boxes = boxes.copy()
        res_boxes[:, [0,2]] = self.image_size - boxes[:, [1,3]]
        res_boxes[:, [1,3]] = boxes[:, [2,0]]
        return res_boxes

class TTACompose(BaseWheatTTA):
    """ author: @shonenkov """
    def __init__(self, transforms):
        self.transforms = transforms
        
    def augment(self, image):
        for transform in self.transforms:
            image = transform.augment(image)
        return image
    
    def batch_augment(self, images):
        for transform in self.transforms:
            images = transform.batch_augment(images)
        return images
    
    def prepare_boxes(self, boxes):
        result_boxes = boxes.copy()
        result_boxes[:,0] = np.min(boxes[:, [0,2]], axis=1)
        result_boxes[:,2] = np.max(boxes[:, [0,2]], axis=1)
        result_boxes[:,1] = np.min(boxes[:, [1,3]], axis=1)
        result_boxes[:,3] = np.max(boxes[:, [1,3]], axis=1)
        return result_boxes
    
    def deaugment_boxes(self, boxes):
        for transform in self.transforms[::-1]:
            boxes = transform.deaugment_boxes(boxes)
        return self.prepare_boxes(boxes)

In [ ]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor

def test_cfg():
    cfg = get_cfg()
    cfg.MODEL.DEVICE='cpu'
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
    #cfg.MODEL.WEIGHTS = '../input/modelfile/model_final.pth'
    cfg.MODEL.WEIGHTS = '../input/modelfile2/model_final _iter10000.pth'
    cfg.DATASETS.TEST = ('wheat_test', )
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.45
    
    return cfg

cfg = test_cfg()
predict = DefaultPredictor(cfg)

In [ ]:
import cv2
def rotate90(img):
    tempimg = cv2.rotate(img.copy(), cv2.ROTATE_90_COUNTERCLOCKWISE)
    outputs = predict(tempimg)['instances']
    boxes = [i.cpu().detach().numpy() for i in outputs.pred_boxes]
    res_boxes = boxes.copy()
    for i in range(len(res_boxes)):
        res_boxes[i] = res_boxes[i].tolist()
        res_boxes[i][0] = 1024 - boxes[i][1]
        res_boxes[i][2] = 1024 - boxes[i][3]
        res_boxes[i][1] = boxes[i][2]
        res_boxes[i][3] = boxes[i][0]
    scores = outputs.scores.cpu().detach().numpy()
    return res_boxes, scores

In [ ]:
def submit():

    for idx, row in tqdm(sub_df.iterrows(), total=len(sub_df)):
        boxes = []
        scores = []
        img_path = os.path.join("../input/global-wheat-detection/test/", row.image_id+'.jpg')
        img = cv2.imread(img_path)
        outputs = predict(img)['instances']
        boxes.extend([i.cpu().detach().numpy() for i in outputs.pred_boxes])
        scores.extend(outputs.scores.cpu().detach().numpy())
        boxes.extend(rotate90(img)[0])
        scores.extend(rotate90(img)[1])
        list_str = []
        for box, score in zip(boxes, scores):
            box[3] -= box[1]
            box[2] -= box[0]
            box = list(map(int, box))
            score = round(score, 4)
            list_str.append(score) 
            list_str.extend(box)
        sub_df.loc[idx, 'PredictionString'] = ' '.join(map(str, list_str))
    
    return sub_df

sub_df = submit()    
sub_df.to_csv('submission.csv', index=False)


In [ ]:
sub_df